## 日本語BERTでlivedoorニュースを相互情報量最大化(IIC)でクラスタリング

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：livedoorニュースをダウンロードして、PyTorchのDataLoaderに変換


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2020-05-21 03:05:54--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.3’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.37MB/s    in 2.5s    

2020-05-21 03:05:58 (3.37 MB/s) - ‘ldcc-20140209.tar.gz.3’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['sports-watch', 'README.txt', 'dokujo-tsushin', 'livedoor-homme', 'peachy', 'smax', 'movie-enter', 'CHANGES.txt', 'it-life-hack', 'kaden-channel', 'topic-news']
カテゴリー数: 9
['sports-watch', 'dokujo-tsushin', 'livedoor-homme', 'peachy', 'smax', 'movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news']


In [0]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [0]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [7]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)


(7376, 2)


In [0]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]


In [0]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)


In [10]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [0]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


- LivedoorニュースをBERT用のDataLoaderにする
- Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [12]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [0]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')


In [0]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [0]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [16]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [0]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


## 準備2：BERTでlivedoorニュースの記事をベクトル化する

In [18]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')


ネットワーク設定完了


In [0]:
# BERTでベクトル化する関数を定義


def vectorize_with_bert(net, dataloader):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloader.batch_size

    # データローダーからミニバッチを取り出すループ
    for index, batch in enumerate(dataloader):
        # batchはTextとLableの辞書オブジェクト
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = batch.Text[0].to(device)  # 文章
        labels = batch.Label.to(device)  # ラベル

        # 順伝搬（forward）計算
        with torch.set_grad_enabled(False):

            # Berに入力
            result = net(inputs)

            # sequence_outputの先頭の単語ベクトルを抜き出す
            vec_0 = result[0]  # 最初の0がsequence_outputを示す
            vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
            vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

            # ベクトル化したデータをtorchリストにまとめる
            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text, vec_0], dim=0)
                list_label = torch.cat([list_label, labels], dim=0)

    return list_text, list_label


In [20]:
# DataLoaderをベクトル化版に変換
# 少し時間がかかります5分弱

list_text_train, list_label_train = vectorize_with_bert(model, dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model, dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model, dl_test)


使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------


In [0]:
# torchのリストをDatasetに変換

from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(
    list_label_train.view(-1, 1), list_text_train)
dataset_bert_eval = TensorDataset(list_label_eval.view(-1, 1), list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1, 1), list_text_test)


In [0]:
# Dataloaderにする
from torch.utils.data import DataLoader

batch_size = 1024

dl_bert_train = DataLoader(
    dataset_bert_train, batch_size=batch_size, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチがbatch_sizeに足りない場合は無視する

dl_bert_eval = DataLoader(
    dataset_bert_eval, batch_size=batch_size, shuffle=False)
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=batch_size, shuffle=False)


## 準備3：IICのディープラーニングモデルを用意

In [0]:
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_RATE = 10


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.Conv1d(1, 400, kernel_size=768, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.Conv1d(1, 300, kernel_size=400, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.Conv1d(1, 300, kernel_size=300, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300, 250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        self.fc2 = nn.Linear(250, 9)

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        self.fc2_overclustering = nn.Linear(250, 9*OVER_CLUSTRING_RATE)

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn2(self.conv2(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x_prefinal = F.relu(self.bnfc1(self.fc1(x)))

        # multi-headは使わず
        y = F.softmax(self.fc2(x_prefinal), dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering


In [0]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        # init.xavier_normal_(m.weight.data)

        # He
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)


In [0]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):
    bn, k = x_out.size()
    assert (x_tf_out.size(0) == bn and x_tf_out.size(1) == k), '{} {} {} {}'.format(
        bn, k, x_tf_out.size(0), x_tf_out.size(1))

    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)  # bn, k, k
    p_i_j = p_i_j.sum(dim=0)  # k, k
    p_i_j = (p_i_j + p_i_j.t()) / 2.  # symmetrise
    p_i_j = p_i_j / p_i_j.sum()  # normalise
    return p_i_j


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # has had softmax applied
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_i_j.size() == (k, k))

    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # avoid NaN losses. Effect will get cancelled out by p_i_j tiny anyway
    # これはPyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS

    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # 参考に、重みalphaを追加

    alpha = 2.0
    loss = (- p_i_j * (torch.log(p_i_j) - alpha *
                       torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss


In [0]:
# データにノイズを加える関数の定義
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim=0).to(device)


def perturb_data(x):
    y = x.clone()
    noise = torch.randn(len(tensor_std)).to(device)*tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y


## 4：IICのネットワークを学習させる

In [0]:
# 学習関数の定義

def train(total_epoch, model, train_loader, optimizer, device):

    # ネットワークを訓練モードに
    model.train()

    # 学習率のスケジューラーCosAnnealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2, eta_min=0)

    for epoch in range(total_epoch):
        for batch_idx, (target, data) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()
            
            data_perturb = perturb_data(data)  # ノイズを与え、変換したデータを作る

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワークへ入れる
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering,
                             output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

        # ログ出力
        if epoch % 50 == 0:
            print('Train Epoch {} \tLoss1: {:.6f} \tLoss2: {:.6f} \tLoss_total: {:.6f}'.format(
                epoch, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer


In [28]:
# 学習の実施(5分弱)

# モデルの用意
net = NetIIC()
net.apply(weight_init)
net.to(device)

# 最適化関数
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) 

total_epoch = 1000

model_trained, optimizer = train(
    total_epoch, net, dl_bert_train, optimizer, device)


Train Epoch 0 	Loss1: -3.942952 	Loss2: -7.741894 	Loss_total: -11.684846
Train Epoch 50 	Loss1: -6.027591 	Loss2: -10.173136 	Loss_total: -16.200727
Train Epoch 100 	Loss1: -6.497630 	Loss2: -11.162491 	Loss_total: -17.660121
Train Epoch 150 	Loss1: -6.564329 	Loss2: -11.766784 	Loss_total: -18.331113
Train Epoch 200 	Loss1: -6.568226 	Loss2: -12.377754 	Loss_total: -18.945980
Train Epoch 250 	Loss1: -6.570720 	Loss2: -12.563309 	Loss_total: -19.134029
Train Epoch 300 	Loss1: -6.559010 	Loss2: -13.132699 	Loss_total: -19.691709
Train Epoch 350 	Loss1: -6.582628 	Loss2: -13.215260 	Loss_total: -19.797887
Train Epoch 400 	Loss1: -6.566440 	Loss2: -13.149175 	Loss_total: -19.715615
Train Epoch 450 	Loss1: -6.562125 	Loss2: -13.176473 	Loss_total: -19.738598
Train Epoch 500 	Loss1: -6.565271 	Loss2: -13.200027 	Loss_total: -19.765298
Train Epoch 550 	Loss1: -6.577297 	Loss2: -13.210943 	Loss_total: -19.788240
Train Epoch 600 	Loss1: -6.568009 	Loss2: -13.191870 	Loss_total: -19.759879
Tra

In [0]:
# モデル分類のクラスターの結果を確認する
import numpy as np

# ミニバッチサイズ1のテスト用のDataLoaderを用意
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=1, shuffle=False)


def test(model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    # 出力用のリストを用意
    total_num = len(test_loader)
    # index, (target_label, inferenced_label)
    output_list = np.zeros((total_num, 2))

    with torch.no_grad():
        for batch_idx, (target, data) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target[0].cpu())

            # 結果をリストにまとめる
            output_list[batch_idx, 0] = target[0][0].cpu()  # 正解ラベル
            output_list[batch_idx, 1] = outputs.argmax(dim=1).cpu()  # 予測ラベル

    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.view(-1, 1).numpy(), ref_targs.numpy(), output_list


In [0]:
# テストデータで推論を実施
out_targs, ref_targs, output_list = test(model_trained, device, dl_bert_test)


In [31]:
# 混同行列（的な）を作る
matrix = np.zeros((9, 9))

# 縦にlivedoorニュースの正解クラスを、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)


[[ 55.   0.   1.   0.   4.  47.   2.  76.   0.]
 [  3.  40.   4.   0.  14.   1.   1.   0. 116.]
 [  7.  39.  21.   4.  16.   3.   6.   3.   1.]
 [ 11.  60.  16.   4.  13.   8.  27.   2.  17.]
 [  8.   6.  20. 107.   1.   8.  16.   0.   1.]
 [ 11.  17.  15.   0.  40.   6.  78.   7.   0.]
 [ 18.   3.  65.  40.  13.  15.  14.   3.   1.]
 [ 63.   7.  45.  11.   2.  42.   7.   1.   1.]
 [ 27.   0.   6.   0.   4.  61.   1.  61.   1.]]


In [32]:
dic_id2cat


{0: 'sports-watch',
 1: 'dokujo-tsushin',
 2: 'livedoor-homme',
 3: 'peachy',
 4: 'smax',
 5: 'movie-enter',
 6: 'it-life-hack',
 7: 'kaden-channel',
 8: 'topic-news'}

In [33]:
# クラスタの結果を確認
#「sports-watch」の5番目のクラスタの文章、7番目のクラスタの文章
#「topic-news」の5番目のクラスタの文章、7番目のクラスタの文章
# を確認して、クラスタ5とクラスタ7の特徴を見てみます。

import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=["正解クラス", "推定クラスタ"]
df2.head()

,正解クラス,推定クラスタ
0,5.0,6.0
1,6.0,0.0
2,3.0,2.0
3,6.0,0.0
4,5.0,6.0


In [34]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==5)].head()

,正解クラス,推定クラスタ
21,0.0,5.0
59,0.0,5.0
126,0.0,5.0
142,0.0,5.0
153,0.0,5.0


In [41]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[21, 0][:300])
print(df.iloc[59, 0][:300])


先月29日、プロレスラーで、その引退後は、選手育成や解説者、レフェリーとしてファンに愛された山本小鉄さんが低酸素脳症で急逝、ファン＆関係者に深い悲しみを与えた。そんな折、今週7日発売の「週刊アサヒ芸能」は、「NEWS SHOT!」のコーナーにおいて、山本さんの“死去直前”の豪傑ぶりをうかがわせる驚くべき行動を報じた。同誌にコメントを寄せた、元週刊プロレス編集長・ターザン山本氏は、「身長170センチに体重113キロの体は、いまだに現役時代を彷彿とさせる筋肉を維持していました。若手同様の過酷なトレーニングをし、68歳とは思えぬ食欲でしたが、実は山本さんは糖尿病だったんです」と明かし、また、先月行わ
豪メディア「world wide of sports」は24日、全豪オープンで日本男子80年ぶりのベスト8進出を果たした錦織圭(にしこり・けい)のインタビューを掲載した。その中で錦織は、ベスト8進出について「初めてのベスト8で、ステップアップした気持ちでいる。だけどプレッシャーは感じていない」とコメント。また対戦相手のアンディ・マレーについては、「リスペクトしているが、彼を倒せるよう頑張る」と話している。ただ、錦織の対戦相手であるマレーは難敵だ。錦織は22歳と若いが、マレーもまだ24歳と若手の部類に入る。そしてマレーは、2008年から現在まで4大大会にて3度の準優勝、現ATPマスターズ1000


In [45]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==7)].head(2)

,正解クラス,推定クラスタ
14,0.0,7.0
18,0.0,7.0


In [47]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[14, 0][:300])
print(df.iloc[18, 0][:300])

「あれで全てが狂った」「ピッチャーのことは触ったこともない」「誰も信用してない」など、とにかく衝撃的な言葉が相次ぎ飛び出した中日ドラゴンズ元監督・落合博満氏のインタビュー。日本テレビ「Going! Sports＆News」では、野球解説者の江川卓氏が聞き手となり、その模様は二夜（17日、18日）に渡って放送された。初回放送分では、「来年ユニフォーム着るってなったら、ここまでは喋らない」とまでいい、知られざるエピソードを明かした落合氏だったが、後編の放送は、これを上回るといっていい更に驚くべき内容となった。以下にインタビューの要約を掲載する。江川：今年は日本一になれると思っていた？思っていなかっ
4日放送、TBS「S1」では、「Weeklyストーブ 猛虎がMLBを語る」と題し、阪神タイガース3選手＝金本知憲、新井貴浩、鳥谷敬の座談会の模様が放送された。冒頭、「全て新井を中心に世界が回っているんで」と、新井を持ち上げた“アニキ”金本だったが、「（新井の）通信簿を付けるとしたら？」と訊かれるや一転、「55点。打率も3割1分から2割6分台、最後。4番ですからね」と厳しく言い放つ。また、海外FA権の取得について、新井は「実際、無理ですね。僕みたいなタイプはまず通じないですね。松井（秀喜）さんでも、ああいう感じですよ。年間50本くらいホームラン打つパワーヒッターですよね」と語り、鳥谷が「自分は何


In [49]:
df2[(df2['正解クラス']==8) & (df2['推定クラスタ']==5)].head(2)

,正解クラス,推定クラスタ
7,8.0,5.0
55,8.0,5.0


In [50]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[7, 0][:300])
print(df.iloc[55, 0][:300])

韓国では一般的に僧侶は独身であることが原則となっているが、「異性からモテず、絶望し出家を決意した」と語る僧侶のFacebookが韓国のネット掲示板で話題になっている。ヒョボンという名前のこの僧侶は、19日、Facebookで「20代の私は異性に人気がありませんでした。（中略）結局どうしようもないという結論に至り、出家を決心しました。皆さんも諦めて出家しなさい。今になって考えれば若いころ、ラップやパンクをすべきだった」と、僧侶になった背景を語っている。また、僧侶生活に関しては「早く出家するほど偉くなるのも早い。偉くなったらご飯と洗濯を悩まなくてもいい」とも語り、ほかにも「アイドルのPVはセクシー
野田首相が10月19日、ソウルで行われた李明博（イミョンバク）大統領との会談にて、韓国への資金支援枠を現行130億ドルの5倍以上の、700億ドル（約5兆3600億円）で拡大する合意をまとめた。韓国大手メディアでは、この日韓の通貨スワップが長期的にウォンの上昇への圧力につながると報道しているものの、あくまで大統領府で首脳会談のひとつとして取り上げたのみ。むしろ李大統領が「歴史の認識を忘れず未来に進むべき」などと野田首相に伝達したことを大きく取り上げている。一方日本では、日韓スワップについてネット上で賛否両論が繰り広げられた。「韓国の外貨不足で日本企業の輸出分の代金回収が出来ず、日本企業の連鎖倒産


In [51]:
df2[(df2['正解クラス']==8) & (df2['推定クラスタ']==7)].head(2)

,正解クラス,推定クラスタ
71,8.0,7.0
76,8.0,7.0


In [52]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[71, 0][:300])
print(df.iloc[76, 0][:300])

18日、日本経済新聞が「被災地もう一つの異常事態 復興特需・原発賠償金・・・マネー流入ゆがむ再生」と題した記事で被災地の歪んだ現実を報じ、ネット掲示板で物議を醸している。同記事では、賠償を受ける対象の家族が五人家族であれば月80万円が懐に入ると指摘し、「東電から金もらって、働かなくてもパチンコしたり、すし食ったりスマホとかばんばん新しくしている」という現地の人の言葉を掲載。この被災者に流れる賠償金とその使い方を「ゆがむ再生」として位置づけている。ネット掲示板では「一切募金しなかったおれに死角はなかったな」「震災さまさまやな」「スマホと寿司は消費だからかまわんのだが、パチンコはダメだって・・・。
今月14日に発売された「週刊文春」（6月21日号）では、AKB48・指原莉乃の元カレを名乗る男性が、交際内容を暴露。指原は翌日の『AKB48のオールナイトニッポン』に生出演し、騒動の釈明と謝罪を行ったが、秋元康氏は、彼女にHKT48への移籍を伝えた。騒動から約2週間——。26日放送のTBS『火曜曲！』では、中居正広が仕掛け人となり、指原本人には生放送と伝えずにその胸中を吐露させるドッキリが行われた。「俺、何にも知らないんだけど」という中居の言葉ではじまった指原とのやり取り。彼女が語った内容とは？以下、指原のコメント（要約）（転勤？）そうです。あの、説明させて頂くと。ちゃんと話した方がいいと思っ


文章を見ただけでは、あまり特徴は分かりませんね。。。

ただ、スポーツとニュースは、記事の雰囲気がとても似ていることは分かります。

これ以上に、IICされたクラスタの特徴をきちんと把握するには、

- wordcloudで単語の頻度の傾向を見てみる
- クラスタ文書の全ベクトルをいじいじして、なんらかクラスタのベクトルを作って、代表文書を決めたり近い単語を出す

などの操作が考えられます。

以上。